In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from thundersvm import SVC
import numpy as np
import cv2

In [ ]:
facenet_model = load_model('facenet_keras.h5')

In [ ]:
def get_embedding_and_label(directory):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        embedding = list()
        # enumerate files
        for filename in listdir(path):
            file_path = directory + filename

            face = cv2.imread(file_path)
            face = cv2.resize(face, (160, 160))
            face = face.astype('float32')

            # standardize pixel values across channels (global)
            mean, std = face.mean(), face.std()
            face = (face - mean) / std
            sample = np.expand_dims(face, axis=0)
            embedding = facenet_model.predict(sample)

            embeddings.append(embedding)

        labels = [subdir for _ in range(len(embeddings))]

        #print('>loaded %d examples for class: %s' % (len(faces), subdir))

        X.extend(embeddings)
        y.extend(labels)
    return asarray(X), asarray(y)

In [ ]:
directory = './'
feature, target = get_embedding_and_label(directory)

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=.2, random_state=101)

encoder = LabelEncoder()
encoder.fit(y_train)

y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

model = SVC(C=100, kernel='rbf')

# Fit the model to training data
model.fit(X_train, y_train)

# Check test set accuracy
accuracy = model.score(X_test, y_test)

print('Accuracy: {}'.format(accuracy))

model.save_to_file('model.txt')